In [1]:
import sys
sys.path.append('..')

In [2]:
from utils import dump_jsonl, load_jsonl, CLOSENESS_LABELS, AUTHORITY_LABELS

In [3]:
ls annotated/v0.1/closeness_v0.1

user0.jsonl  user1.jsonl  user2.jsonl  user3.jsonl


In [4]:
import pandas as pd

users = ["user0", "user1", "user2", "user3"]

def authority_to_degree(label):
    if label == 'B ให้เกียรติ':
        return 3
    elif label == 'B ทำตัวปกติ':
        return 2
    elif label == 'B ไม่ให้เกียรติกัน':
        return 1
    else:
        return None

def closeness_to_degree(label):
    if label == 'สนิทกันมาก':
        return 4
    elif label == 'แค่คนรู้จักกัน':
        return 3
    elif label == 'ไม่รู้จักกัน':
        return 2
    elif label == 'ไม่ชอบหน้ากัน':
        return 1
    else:
        return None
    
    
def load_data(filename, column_name, to_degree):
    annotated = load_jsonl(filename)
    
    _df = []
    for row in annotated:
        if len(row["label"])==0:
            continue

        _df.append({
            "text": row["text"],
            "tweet": row["tweet"],
            column_name: row["label"][0],
            f"{column_name}_degree": to_degree(row["label"][0])
        })
    
    return pd.DataFrame(_df)

In [5]:
from collections import defaultdict

# ordinal_weights
def get_weights(categories):
    weights = defaultdict(dict)
    if len(categories)==3:
        mat = [[1.00, 0.67, 0.00], 
               [0.67, 1.00, 0.67], 
               [0.00, 0.67, 1.00]]
    elif len(categories)==4:
        mat = [[1.00, 0.83, 0.50, 0.00], 
               [0.83, 1.00, 0.83, 0.50], 
               [0.50, 0.83, 1.00, 0.83], 
               [0.00, 0.50, 0.83, 1.00]]
    elif len(categories)==5:
        mat = [[1, 0.9, 0.7, 0.4, 0.0], 
               [0.9, 1, 0.9, 0.7, 0.4], 
               [0.7, 0.9, 1, 0.9, 0.7], 
               [0.4, 0.7, 0.9, 1, 0.9],
               [0.0, 0.4, 0.7, 0.9, 1]]
    else:
        # Lazy to implement in case of len(categories) > 5
        raise Exception("No Implementation")
        
    for i, l in enumerate(categories): 
        for j, k in enumerate(categories): 
            weights[l][k] = mat[i][j]
    return weights

    
def cal_agreement(df1, df2, column, categories, cat_column):
    merged = pd.merge(df1, df2, on=column)
#     assert(len(df1)==len(merged))
    merged = merged.dropna()    
    cnt_matrix = defaultdict(dict)
    acc_matrix = defaultdict(dict)
    
    for l in categories: 
        for k in categories: 
            d = merged
            d = d[d[f"{cat_column}_x"]==k]
            d = d[d[f"{cat_column}_y"]==l]
            cnt_matrix[l][k] = len(d)
    
    for l in categories: 
        d = merged
        d = d[d[f"{cat_column}_x"]==l]
        acc_matrix["x"][l] = len(d)
        
        d = merged
        d = d[d[f"{cat_column}_y"]==l]
        acc_matrix["y"][l] = len(d)
    
    weights = get_weights(categories)
    
    N = len(merged)
    Pa = 0
    for l in categories: 
        for k in categories: 
            Pa += weights[l][k]*cnt_matrix[l][k]/N
    
    Pe = 0
    for l in categories: 
        for k in categories: 
            Pe += weights[l][k]*(acc_matrix["x"][l]/N)*(acc_matrix["y"][k]/N)
    
    if Pe==1:
        raise Exception("Divide by zero")
    
    kappa = (Pa-Pe)/(1-Pe)
    return kappa

# Test Agreement

In [6]:
auth_df = {}
clos_df = {}
for u in users:
    auth_df[u] = load_data(f"annotated/v0.1/authority_v0.1/{u}.jsonl", "authority", authority_to_degree)
    clos_df[u] = load_data(f"annotated/v0.1/closeness_v0.1/{u}.jsonl", "closeness", closeness_to_degree)

Loaded 20 records from annotated/v0.1/authority_v0.1/user0.jsonl
Loaded 20 records from annotated/v0.1/closeness_v0.1/user0.jsonl
Loaded 20 records from annotated/v0.1/authority_v0.1/user1.jsonl
Loaded 20 records from annotated/v0.1/closeness_v0.1/user1.jsonl
Loaded 20 records from annotated/v0.1/authority_v0.1/user2.jsonl
Loaded 20 records from annotated/v0.1/closeness_v0.1/user2.jsonl
Loaded 20 records from annotated/v0.1/authority_v0.1/user3.jsonl
Loaded 20 records from annotated/v0.1/closeness_v0.1/user3.jsonl


In [7]:
import numpy as np

In [10]:
auth_df["user0"]

,text,tweet,authority,authority_degree
0,A: #คลับเฮ้าส์toxic ไฮโซแบบนี้ เราก็มำได้นะ เผ...,https://twitter.com/i/web/status/1456862455305...,B ทำตัวปกติ,2
1,A: หนูมีพี่คนหนึ่งเป็นไอดอลค่ะแม่ หนูจะขายหีเห...,https://twitter.com/i/web/status/1488876213754...,B ไม่ให้เกียรติกัน,1
2,A: ความคิดเห็นส่วนตัว : #คลับเฮาส์toxic มีคำพู...,https://twitter.com/i/web/status/1457174314113...,B ให้เกียรติ,3
3,A: สีม่วงคือข้อมูลจากสเปซวันนี้ สีน้ำเงินคือที...,https://twitter.com/i/web/status/1494612254544...,B ให้เกียรติ,3
4,A: เขินเวกัสจูบพีททุกครั้งเลยอ่ะ นังชอบใช้จมูก...,https://twitter.com/i/web/status/1546146399682...,B ให้เกียรติ,3
5,A: อยากมาแชร์ปสก.นิดหน่อย เราเคยกดดันตัวเองมาก...,https://twitter.com/i/web/status/1520391472457...,B ให้เกียรติ,3
6,A: ฟังไปจดไปยังงงเลย 🤔🤔 #หัวหน้าวงที่ขึ้นต้นว่...,https://twitter.com/i/web/status/1494319446323...,B ทำตัวปกติ,2
7,A: รำคาญพวกที่บ่นว่า สังคมได้ประโยชน์อะไรจากกา...,https://twitter.com/i/web/status/1500489552485...,B ทำตัวปกติ,2
8,A: ความอำมหิตวงการบันเทิงสำหรับพระสึกใหม่ - อิ...,https://twitter.com/i/web/status/1487700593460...,B ทำตัวปกติ,2
9,A: #DSIแตงโม อีกเรื่องหนึ่งที่สงสัย เวลาตกกับเ...,https://twitter.com/i/web/status/1509109934428...,B ทำตัวปกติ,2


In [8]:
print("Authority")
alliaa = []
for u in users[1:]:
    iaa = cal_agreement(auth_df["user0"], auth_df[u], column="tweet", categories=[1,2,3], cat_column="authority_degree")
    print(u, iaa)
    alliaa.append(iaa)
    
print(alliaa, np.mean(alliaa))

Authority
user1 0.5152998776009788
user2 0.5152998776009788
user3 0.5119356512714061
[0.5152998776009788, 0.5152998776009788, 0.5119356512714061] 0.5141784688244546


In [9]:
print("Closeness")
alliaa = []
for u in users[1:]:
    iaa = cal_agreement(clos_df["user0"], clos_df[u], column="tweet", categories=[1,2,3,4], cat_column="closeness_degree")
    print(u, iaa)
    alliaa.append(iaa)
    
print(alliaa, np.mean(alliaa))

Closeness
user1 0.6161425159235672
user2 0.6734563971992363
user3 0.6373333333333332
[0.6161425159235672, 0.6734563971992363, 0.6373333333333332] 0.6423107488187122
